Connor Krill July 22, 2019
# Visualize the optimization progress as a movie
Plots lagranian distance vs variable and animates it over the entire simulation time.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import os
import numpy as np
import sys
sys.path.append("../tools")
from hyades_output_reader import createOutput

plt.style.use('ggplot')

def animate(i):
    '''Updates the information shown on the axis.
        Relies on externally defined variables.'''
    if int(objects[0].time[i])%1 == 0: # update time 
        time_text = f'Time: {objects[0].time[i]:.0f} ns'
        my_text._text = time_text
#         ax1.set_title(time_text, fontsize=32)
    for line, obj in zip(lines, objects): # update each line serperately
        ydata = obj.output[:len(obj.X),i]
        line.set_ydata(ydata)
        
    return lines

# Setup - Only need to edit the cell below. 
Adjustments for figure lables and movie properities can be made further down in the code.

In [2]:
path = '/Users/krill2/Documents/data/RFS/'
run_names = ['CH_MgO_Quartz_Hydro']

save_movie = False
out_fname = 'Mgo Shock Pres+U'

title = 'MgO Shock Simulation'

### Everything below is optional
# if None then X defaults to entire sample and ylimits to maximum value for that variable
xlim = None # (min, max)
pressure_ylim    = (0, 500)
temperature_ylim = (0, 30)

label_material = 'MgO'

In [4]:
pres = [createOutput(os.path.join(path, f, f), 'Pres') for f in run_names]
temp = [createOutput(os.path.join(path, f, f), 'U') for f in run_names]
objects = pres + temp

frame_num = len(objects[0].time) # the number of time steps is the number of frames
# frame_num = 200

# intialize and format a figure/axis
fig, ax1 = plt.subplots(figsize=(10,6))
ax2 = ax1.twinx()
if xlim:
    ax1.set(xlim=xlim)
if pressure_ylim:
    ax1.set(ylim=pressure_ylim)
if temperature_ylim:
    ax2.set(ylim=temperature_ylim)
    
# intialize all the lines on the axis
lines = []
for P, T, name in zip(pres, temp, run_names):
    L1 = ax1.plot(P.X, P.output[:, 0],  'b', lw=4,  label='Pressure')[0]
    lines.append(L1)
    L2 = ax2.plot(T.X, T.output[:len(T.X), 0],  'r', lw=4, label='Particle Velocity')[0]
    lines.append(L2)

# Add time text
x = ax1.get_xlim()[1]
y = ax1.get_ylim()[1]
my_text = ax1.text(x * 0.99, y * 0.8, 'Time: 0 ns', fontsize=32, ha='right')
    
# format the plot
ax1.set_title(title,)
ax1.set_xlabel('Distance (μm)', fontsize=24)
ax1.set_ylabel('Pressure (GPa)', fontsize=24, color='b')
ax2.grid()
ax2.set_ylabel('Velocity (km/s)', fontsize=24, color='r', rotation=-90, labelpad=30)
ax1.legend(loc=2, fontsize='large')
ax2.legend(loc=1, fontsize='large')

ax1.tick_params(axis='y', colors='b')
ax2.tick_params(axis='y', colors='r')
ax1.tick_params(axis='both', labelsize=20)
ax2.tick_params(axis='y', labelsize=20)
###

# add filled region representing the material of interest
if label_material:
    x = [pres[0].material_properties[label_material]['startX'],
         pres[0].material_properties[label_material]['endX']]
    y = [ax1.get_ylim()[1]] * 2
    ax1.fill_between(x, y, color='b', alpha=0.25)
    ax1.text((x[0] + x[1])/2, ax1.get_ylim()[1]*0.9, label_material, fontsize=24, ha='center')

# call the animator
anim = animation.FuncAnimation(fig, animate,
                               frames=frame_num, interval=20,
                               repeat_delay=2000, blit=True)
plt.close() # close figure automatically generated by the animiation call
fig.tight_layout()

# optional saving
if save_movie:
    print('Saving...')
    out_path = './'
    Writer = animation.writers['ffmpeg']
    writer = Writer(fps=32, metadata=dict(artist='Me'), bitrate=1800)
    anim.save(os.path.join(out_path, out_fname+'.mp4'), dpi=200, writer=writer)
    print('Saved {}'.format(out_fname))
else:
    print('Ready for display')

Saving...
Saved Mgo Shock Pres+U


# Display the movie here is usually much lower quality than saving and watching the .mp4

In [5]:
HTML(anim.to_html5_video()) 